In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import os
import pandas as pd

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM
import keras as kr

from keras import layers
from keras import models
from keras.utils import np_utils

## Load Data
Data has been normalized by subject.

In [ ]:
#load train test split
train = np.load('sttrainingindices.npy')
test = np.load('sttestingindices.npy')

In [ ]:
#load all data
DLMO = np.load('DLMO.npy') #file containing true DLMO
zcmfeats = np.load('zcmfeats.npy') #file containing 24 h of minute by minute activity data for each subject in zero crossings mode
lightfeatsavg = np.load('lightfeatsavg.npy') #file containing 24 h of minute by minute light data for each subject

#split into train and test sets
DLMOtrain = DLMO[train]
DLMOtest = DLMO[test]

lightmaxtest = lightfeatsmax[test]
lightmaxtrain = lightfeatsmax[train]
lightavgtest = lightfeatsavg[test]
lightavgtrain = lightfeatsavg[train]
zcmtest = zcmfeats[test]
zcmtrain = zcmfeats[train]

## Make Classification Problem

In [ ]:
def getidx(dlmo):
    '''get index just before where DLMO occurs'''
    dlmo = dlmo % 24
    inds30 = np.arange(0, 24, .5)
    inds15 = np.arange(0, 24, .25)
    before = np.where(inds30<dlmo)[0][-1]
    before15 = np.where(inds15<dlmo)[0][-1]
    return before, before15

In [ ]:
def builddata(idx, data):
    '''build classification set with wrapped data from 6 h before and 6 h after DLMO'''
    numidx = int(6*len(data)/24) #total number of indices to shift in each direction for 6h centered data
    newdata = np.zeros((2*numidx, len(data)))
    datawrap = np.concatenate((data, data, data))
    for i in range(-numidx+idx+1, numidx+idx+1):
        if i<0:
            newdata[i+numidx-idx-1, :] = np.concatenate((data[i:], data[:i+len(data)]))
        else:
            newdata[i+numidx-idx-1, :] = datawrap[i:i+len(data)]
    return newdata

In [ ]:
#build datasets
def buildclassificationset(dataset, training = True, fifteen = False):
    subs, feats = np.shape(dataset)
    numidx = int(6*feats/24)
    subjclass = np.zeros((1, feats))
    for (i, data) in enumerate(dataset):
        if training:
            dlmo = DLMOtrain[i]
        else:
            dlmo = DLMOtest[i]
        idx = getidx(dlmo)[1*fifteen]
        subjclass = np.concatenate((subjclass, builddata(idx, data)))
    subjclass = subjclass[1:]
    return subjclass

In [ ]:
#classification datasets for periodic features
zcmclasstrain = buildclassificationset(zcmtrain)
zcmclasstest = buildclassificationset(zcmtest, training=False)
lightclasstrain = buildclassificationset(lightavgtrain)
lightclasstest = buildclassificationset(lightavgtest, training=False)

In [ ]:
def replicate(dataset, n):
    return np.repeat(dataset, n, axis = 0)

In [ ]:
#make class vector
subjclass = np.concatenate((np.zeros(12), np.ones(12)))
outclasstrain = np.tile(subjclass, len(DLMOtrain))
outclasstest = np.tile(subjclass, len(DLMOtest))

## Set Up Cross Validation

In [ ]:
#combine to make activity and light dataset
trainzcmlight = np.concatenate((zcmclasstrain, lightclasstrain), axis=1)
testzcmlight = np.concatenate((zcmclasstest, lightclasstest), axis=1)

In [ ]:
def getCV(traindata, trainout):
    featsholdout = np.array_split(traindata, 10)
    outsholdout = np.array_split(trainout, 10)
    featsin = [None]*10
    outsin = [None]*10
    for j in range(10):
        for k in range(10):
            if k != j:
                if np.shape(featsin[j])==():
                    featsin[j] = featsholdout[k]
                    outsin[j] = outsholdout[k]
                else:
                    featsin[j] = np.concatenate((featsin[j], featsholdout[k]), axis = 0)
                    outsin[j] = np.concatenate((outsin[j], outsholdout[k]), axis = 0)
    return featsholdout, outsholdout, featsin, outsin

In [ ]:
#get crossvalidation splits for each data set
valinszcmlight, valoutszcmlight, traininszcmlight, trainoutszcmlight = getCV(trainzcmlight, outclasstrain)
valinszcm, valoutszcm, traininszcm, trainoutszcm = getCV(zcmclasstrain, outclasstrain)

## Two Layer Model with Dropout

In [ ]:
def twolayerdropout(nodes1, nodes2, p, trainin, trainout, valin, valout, epochs, epochtest, method, test=False):
    insize = np.shape(trainin)[1]
    model = models.Sequential()
    model.add(Dropout(p, input_shape = (insize,)))
    model.add(Dense(nodes1, activation = 'relu'))
    model.add(Dropout(p))
    model.add(Dense(nodes2, activation = 'relu'))
    model.add(Dropout(p))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer=method, loss='binary_crossentropy', metrics = ['accuracy'])
    if epochtest:
        history = model.fit(trainin, trainout, epochs = epochs, validation_data=(valin,valout), verbose = 0)
        return history.history['loss'], history.history['val_loss']
    model.fit(trainin, trainout, epochs = epochs, verbose = 0)
    if test:
        return model.predict(valin), model.evaluate(valin, valout, verbose=0)[1]
    return model.evaluate(valin, valout, verbose=0)[1]

### Testing

In [ ]:
outszcmlightDLD, perfzcmlightDLD = twolayerdropout(40, 40, .4, trainzcmlight, outclasstrain, testzcmlight, outclasstest, 100, False, kr.optimizers.sgd(lr=.001), test=True)
print(perfzcmlight)

In [ ]:
outszcmlightDLDtrain, perfzcmlightDLDtrain = twolayerdropout(40, 40, .4, trainzcmlight, outclasstrain, trainzcmlight, outclasstrain, 100, False, kr.optimizers.sgd(lr=.001), test=True)
print(perfzcmlightDLDtrain)

# Error Analysis

In [ ]:
#get under/over values for DLMO switch times
def overunder(num): #get distance over/under .5
    if num>.5:
        num = num -.5
    zero_error = abs(.25-num)
    over_error = .25-num
    under_error = num-.25
    return zero_error, over_error, under_error

#convert to decimal
DLMOtrain_dec = DLMOtrain%1
DLMOtest_dec = DLMOtest%1

trainovers = np.zeros(len(DLMOtrain))
trainunders = np.zeros(len(DLMOtrain))
trainzeros = np.zeros(len(DLMOtrain))
testovers = np.zeros(len(DLMOtest))
testunders = np.zeros(len(DLMOtest))
testzeros = np.zeros(len(DLMOtest))
for i, num in enumerate(DLMOtrain_dec):
    z, o, u = overunder(num)
    trainovers[i] = o
    trainunders[i] = u
    trainzeros[i] = z
    
for i, num in enumerate(DLMOtest_dec):
    z, o, u = overunder(num)
    testovers[i] = o
    testunders[i] = u
    testzeros[i] = z

In [ ]:
def getswitch(outs):
    outs = np.round(outs).reshape((-1, 24)).T
    predswitch = []
    for i in range(np.shape(outs)[1]):
        predswitch.append(np.where(outs[:, i]==1)[0][0])
    return np.array(predswitch)

In [ ]:
def get_error(indexoutput, correctzero, correctunder, correctover):
    index_error = indexoutput - 12 #correct prediction is 12
    errors = np.zeros(len(indexoutput))
    for i, e in enumerate(index_error):
        if e == 0:
            errors[i] = correctzero[i]
        elif e>0:
            errors[i] = .5*e+correctover[i]
        else:
            
            errors[i] = .5*np.abs(e)+correctunder[i]
    return errors

In [ ]:
def get_signed_error(indexoutput, correctzero, correctunder, correctover):
    index_error = indexoutput - 12 #correct prediction is 12
    errors = np.zeros(len(indexoutput))
    for i, e in enumerate(index_error):
        if e == 0:
            errors[i] = correctzero[i]
        elif e>0:
            errors[i] = .5*e+correctover[i]
        else:
            
            errors[i] = -1*(.5*np.abs(e)+correctunder[i])
    return errors

In [ ]:
#get errors from different models
#for testing set
predstest = [outszcmlightDLD]

errors = np.zeros((len(DLMOtest), len(predstest)))

for i, p in enumerate(predstest):
    pred_index = getswitch(p)
    errors[:, i] = get_signed_error(pred_index, testzeros, testunders, testovers)


#for training set
predstrain = [outszcmlightDLDtrain]

errorstrain = np.zeros((len(DLMOtrain), len(predstrain)))

for i, p in enumerate(predstrain):
    pred_index = getswitch(p)
    errorstrain[:, i] = get_signed_error(pred_index, trainzeros, trainunders, trainovers)
    
predictedDLMOtrain = np.zeros((len(DLMOtrain), len(predstrain)))
predictedDLMOtest = np.zeros((len(DLMOtest), len(predstest)))

for i, p in enumerate(predstrain):
    predictedDLMOtrain[:, i] = DLMOtrain + errorstrain[:, i]
    predictedDLMOtest[:, i] = DLMOtest + errors[:, i]

In [ ]:
def get_RMSE(errors):
    methodRMSE = np.zeros(np.shape(errors)[1])
    for i in range(np.shape(errors)[1]):
        methodRMSE[i] = np.sqrt(np.mean(errors[:, i]**2))
    return methodRMSE
        

In [ ]:
RMSEs = get_RMSE(errors)

In [ ]:
accuracies = 100*np.array([doubleDO_acc])

In [ ]:
predictedDLMOtrain = np.zeros((len(DLMOtrain), len(predstrain)))
predictedDLMOtest = np.zeros((len(DLMOtest), len(predstest)))

for i, p in enumerate(predstrain):
    predictedDLMOtrain[:, i] = DLMOtrain + errorstrain[:, i]
    predictedDLMOtest[:, i] = DLMOtest + errors[:, i]